# Deep Deterministic Policy Gradients (DDPG)

Homework project for DRL Continuous Action.

### 1. Import the Necessary Packages

In [ ]:
import gym
import random
import torch
import numpy as np
from collections import deque
from unityagents import UnityEnvironment
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

### 2. Instantiate the Environment and Agent

#### 2.1 Start the Environment

Before running the code cell below, change the file_name parameter to match the location of the Unity environment that you downloaded.

- Mac: "unity_env_single_agent/Reacher.app"
- Windows (x86): "unity_env_single_agent/Reacher_Windows_x86/Reacher.exe"
- Windows (x86_64): "unity_env_single_agent/Reacher_Windows_x86_64/Reacher.exe"
- Linux (x86): "unity_env_single_agent/Reacher.x86"
- Linux (x86_64): "unity_env_single_agent/Reacher.x86_64"
- Linux (x86, headless): "unity_env_single_agent/Reacher_Linux_NoVis/Reacher.x86"
- Linux (x86_64, headless): "unity_env_single_agent/Reacher_Linux_NoVis/Reacher.x86_64"

For instance, if you are using a Mac, then you downloaded Reacher.app. If this file is in the same folder as the notebook, then the line below should appear as follows:

env = UnityEnvironment(file_name="Reacher.app")

In [ ]:
env = UnityEnvironment(file_name='unity_env_single_agent/Reacher.x86_64')

Environments contain brains which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

#### 2.2 Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of +0.1 is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of 33 variables corresponding to position, rotation, velocity, and angular velocities of the arm. Each action is a vector with four numbers, corresponding to torque applicable to two joints. Every entry in the action vector must be a number between -1 and 1.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Train the Agent with DDPG

#### 3.1 Instantiate Agent with DDPG

In [ ]:
from ddpg_agent import Agent

agents = [Agent(state_size=state_size, action_size=action_size, random_seed=12) for _ in range(num_agents)]

Few utilitity functions:

In [ ]:
def do_all(iterable, func):
    for i in iter(iterable):
       func(i)

In [ ]:
def get_SARs(env_info):
    next_state = env_info.vector_observations
    reward = env_info.rewards
    done = env_info.local_done
    return (next_state, reward, done)

In [ ]:
def ddpg(n_episodes=2000, max_t=700):
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        do_all(agents, lambda agent: agent.reset())
        score = 0
        for t in range(max_t):
            actions = [agents[i].act(states[i]) for i in range(len(env_info.agents))]
            env_info = env.step(actions)[brain_name]
            next_states, rewards, dones = get_SARs(env_info)
            [agents[i].step(states[i], actions[i], rewards[i], next_states[i], dones[i]) for i in range(len(env_info.agents))]
            
            states = next_states
            score += np.mean(rewards)
            if sum(dones) > 0:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % 100 == 0:
            torch.save(agents[0].actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agents[0].critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 4. Watch a Smart Agent

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

for i in range(3):
    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = agents.act(state, 0., reward)            # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### 5. Cleanup

When finished, you can close the environment.

In [ ]:
env.close()